In [2]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# 필수 라이브러리 설치 (필요 시)
!pip install aicrowd-cli

# API 키 입력
# ' '안에 AIcrowd 개인 api 키 복붙하기'
API_KEY = '726ac948e26c96a9d34be4478ce98829'

# 로그인
!aicrowd login --api-key {API_KEY}

# 데이터셋 다운로드 (한 번만 실행)
!aicrowd dataset download --challenge mapping-challenge-old

# 압축 해제용 폴더 생성
!mkdir -p /content/mapping-challenge-old/train
!mkdir -p /content/mapping-challenge-old/val
!mkdir -p /content/mapping-challenge-old/test_images

# 압축 해제 (파일 경로가 맞는지 확인 필요)
!tar -xvzf /content/train.tar.gz -C /content/mapping-challenge-old/train
!tar -xvzf /content/val.tar.gz -C /content/mapping-challenge-old/val
!tar -xvzf /content/test_images.tar.gz -C /content/mapping-challenge-old/test_images

print("데이터 다운로드 및 압축 해제 완료")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
test_images/000000057916.jpg
test_images/000000041787.jpg
test_images/000000009802.jpg
test_images/000000024091.jpg
test_images/000000007565.jpg
test_images/000000033395.jpg
test_images/000000053265.jpg
test_images/000000042010.jpg
test_images/000000036346.jpg
test_images/000000021074.jpg
test_images/000000000271.jpg
test_images/000000042658.jpg
test_images/000000055942.jpg
test_images/000000038738.jpg
test_images/000000031431.jpg
test_images/000000037690.jpg
test_images/000000023709.jpg
test_images/000000028410.jpg
test_images/000000030479.jpg
test_images/000000052855.jpg
test_images/000000009680.jpg
test_images/000000031256.jpg
test_images/000000039727.jpg
test_images/000000004322.jpg
test_images/000000023903.jpg
test_images/000000034726.jpg
test_images/000000057275.jpg
test_images/000000006252.jpg
test_images/000000000274.jpg
test_images/000000043786.jpg
test_images/000000034841.jpg
test_images/000000026485.jpg
test_images/000000006934.jpg
test_im

In [7]:
import os
import json
import torch
import numpy as np
from PIL import Image
from shapely.geometry import Polygon
from rasterio import features
from tqdm import tqdm

# 전처리용 입력 파일 경로 및 이미지 경로 설정 (실제 폴더 및 파일 확인 필요)
ANNOTATION_JSON = '/content/mapping-challenge-old/train/train/annotation.json'
IMAGES_DIR = '/content/mapping-challenge-old/train/train/images'

# 전처리 결과 저장 경로 (구글 드라이브 하위, 영구저장)
OUTPUT_DIR = '/content/drive/MyDrive/mapping_challenge_final_dataset'
os.makedirs(os.path.join(OUTPUT_DIR, 'images'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels_dict'), exist_ok=True)

MAX_SAMPLES = 2000    # 실제 처리할 데이터 개수 제한 (속도, 메모리 관리 목적)
TILE_SIZE = 256       # 최종 이미지 타일 크기 설정

# ----------------------------------------
# [헬퍼 함수]
# 라벨 마스크(인스턴스 ID로 칠해진 2D 배열)를 딥러닝 모델 학습 시 필요한
# 'targets' 딕셔너리 형식(마스크와 클래스 정보)으로 변환합니다.
def convert_to_target_format(label_mask):
    # 마스크 내 고유 인스턴스 ID (배경 0은 제외)
    instance_ids = torch.unique(label_mask)
    instance_ids = instance_ids[instance_ids != 0]
    if len(instance_ids) == 0:
        # 건물이 없는 경우, 빈 딕셔너리 반환해 학습 오류 방지
        return {'instance_class': torch.tensor([], dtype=torch.long),
                'masks': torch.tensor([], dtype=torch.uint8)}
    # 각 인스턴스별 바이너리 마스크 생성 (N x H x W)
    masks = (label_mask == instance_ids[:, None, None])
    # 모든 인스턴스는 '건물' 클래스(0)로 할당
    instance_classes = torch.zeros(len(instance_ids), dtype=torch.long)
    return {'instance_class': instance_classes, 'masks': masks.to(torch.uint8)}

# ----------------------------------------
# [헬퍼 함수]
# COCO polygon 어노테이션 리스트를 받아 이미지 크기에 맞는 라벨 마스크로 변환합니다.
def polygons_to_mask(polygons, height, width):
    mask = np.zeros((height, width), dtype=np.int32)
    for idx, polygon_coords in enumerate(polygons, 1):
        poly = Polygon(polygon_coords)
        if not poly.is_valid:
            poly = poly.buffer(0)  # 폴리곤 오류 보정
        if not poly.is_empty:
            rasterized = features.rasterize([(poly, idx)], out_shape=(height, width))
            mask = np.maximum(mask, rasterized)  # 중첩 마스킹 처리
    return mask

# ----------------------------------------
# COCO json 어노테이션 로드 및 이미지별 어노테이션 매핑
with open(ANNOTATION_JSON, 'r') as f:
    coco = json.load(f)

image_id_to_anns = {}
for ann in coco['annotations']:
    image_id_to_anns.setdefault(ann['image_id'], []).append(ann)

# ----------------------------------------
# 메인 전처리 루프: 각 이미지에 대해
# 1) 전체 이미지 크기 마스크 생성
# 2) 256x256 크기로 여러 개 타일링 후 타일별 이미지, 타겟 저장
for img_info in tqdm(coco['images'][:MAX_SAMPLES]):
    img_path = os.path.join(IMAGES_DIR, img_info['file_name'])
    image = Image.open(img_path).convert('RGB')
    width, height = image.size

    # 해당 이미지 어노테이션 폴리곤 추출
    anns = image_id_to_anns.get(img_info['id'], [])
    polygons = []
    for ann in anns:
        seg = ann.get('segmentation', [])
        if isinstance(seg, list) and len(seg) > 0:
            if isinstance(seg[0], (list, tuple)):
                for poly in seg:
                    if len(poly) >= 6 and len(poly) % 2 == 0:
                        points = [(poly[i], poly[i+1]) for i in range(0, len(poly), 2)]
                        polygons.append(points)
            else:
                if len(seg) >= 6 and len(seg) % 2 == 0:
                    points = [(seg[i], seg[i+1]) for i in range(0, len(seg), 2)]
                    polygons.append(points)

    if len(polygons) == 0:
        continue  # 어노테이션 폴리곤 없으면 처리 제외

    # 전체 크기 마스크 생성
    label_mask = polygons_to_mask(polygons, height, width)

    # 256x256 이미지 및 마스크 타일 단위로 분할 처리
    for top in range(0, height, TILE_SIZE):
        for left in range(0, width, TILE_SIZE):
            bottom = min(top + TILE_SIZE, height)
            right = min(left + TILE_SIZE, width)

            # 크기가 부족한 가장 자리 부분 스킵
            if (bottom - top) < TILE_SIZE or (right - left) < TILE_SIZE:
                continue

            img_tile = np.array(image)[top:bottom, left:right, :]
            mask_tile = label_mask[top:bottom, left:right]

            # 타일 마스크로 타겟 딕셔너리 생성
            target = convert_to_target_format(torch.from_numpy(mask_tile).long())
            # 타일 이미지 배열을 (C,H,W) 텐서로 변환
            image_tensor = torch.from_numpy(img_tile).permute(2, 0, 1).float()

            tile_id = f"{os.path.splitext(img_info['file_name'])[0]}_{top}_{left}"
            # 저장 경로 지정
            img_save_path = os.path.join(OUTPUT_DIR, 'images', f"{tile_id}.pt")
            target_save_path = os.path.join(OUTPUT_DIR, 'labels_dict', f"{tile_id}.pt")

            # 이미지 타일과 타겟 딕셔너리 별도 저장
            torch.save(image_tensor, img_save_path)
            torch.save(target, target_save_path)

print(f"총 {MAX_SAMPLES}개 샘플에 대해 데이터 전처리 및 저장 완료!")


100%|██████████| 2000/2000 [01:47<00:00, 18.66it/s]

총 2000개 샘플에 대해 데이터 전처리 및 저장 완료!
